In [2]:
import os
import json
import json
import regex as re
from tqdm.auto import tqdm
tqdm.pandas()
import math
import pandas as pd
import string
# from pyvi.ViTokenizer import tokenize
import pandas as pd
import numpy as np
import json, pickle
from rank_bm25 import BM25Okapi
import argparse
import gc
from tqdm.auto import tqdm
tqdm.pandas()
import regex as re
import string

import json 
from glob import glob 
import re 
from nltk import word_tokenize as lib_tokenizer 
import string

from pandarallel import pandarallel
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import numpy as np
pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=10)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
def get_topk(query, topk = 100):
    tokenized_query = query.split()
    tfidf_query = tfidf_model[dictionary.doc2bow(tokenized_query)]
    scores = bm25_index[tfidf_query]
    top_n = np.argsort(scores)[::-1][:topk]
    texts = [df.description.values[i] for i in top_n]
    objectives = [df.objective.values[i] for i in top_n]
    # print(titles)
    # print(tfidf_query, scores)
    return objectives, texts, scores[top_n]

def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    x = "".join([i for i in x if i not in string.punctuation])
    return x

dict_map = dict({})  
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 
 
def strip_answer_string(text): 
    text = text.strip() 
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] != '(' and text[-1] == ')' and '(' in text: 
            break 
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1: 
            break 
        text = text[:-1].strip() 
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1: 
            break 
        text = text[1:].strip() 
    text = text.strip() 
    return text 
 
def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

In [4]:
df_path = "./final_objectives_cleaned.csv"
df = pd.read_csv(df_path)

In [5]:
df = df.fillna("NaN")

In [6]:
df['description'] = df['description'].apply(lambda x: x.lower()).parallel_apply(post_process)
corpus = [x.split() for x in df['description'].values]
dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')  # Enforce binary weighting of queries
dictionary.save("./outputs/bm25_description/dict")
tfidf_model.save("./outputs/bm25_description/tfidf")
bm25_index.save("./outputs/bm25_description/bm25_index")

In [7]:
# get_topk("This summarizes the organization's adherence to relevant regulations and standards.", topk=10)
get_topk("abstract", topk=10)

(['metaphysical poetry',
  'political thriller',
  'Harold Pinter',
  "Eugene O'Neill",
  'Bertolt Brecht',
  'Neil Simon',
  'Edward Albee',
  'Tom Stoppard',
  'Lorraine Hansberry',
  'August Wilson'],
 ['this style is explores the realm of abstract concepts and spiritual truths through vivid imagery and symbolism ',
  'this style is is a genre of fiction that blends political intrigue with suspenseful storytelling  often focusing on corruption  espionage  and highstakes political maneuvering ',
  'the author s writing style is characterized by sharp  concise sentences and a minimalist use of language  often punctuated by pauses and silences  his style is characterized by a sense of unease and mystery  often exploring themes of memory  identity  and the human condition ',
  'the author s writing style is characterized by lyrical  poetic language  often laced with humor and social commentary  he uses vivid imagery and symbolism to explore themes of human suffering  resilience  and ide